In [5]:
import re
import sys
import gc
import datetime
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup as soup
from bs4 import Comment
from urllib.request import urlopen as uReq
from references_dict import Team_Dictionary,TableColumns

In [21]:
# get soup object from link
def get_soup(link):
    uClient = uReq(link)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "lxml")
    return page_soup       

# utility method for parsing game page tables
def get_data(page_soup,id,commented=0):
    data = page_soup.find("div",{"id":id})
    if commented > 0:
        comment = data.find(string=lambda text:isinstance(text,Comment))
        data = soup(comment,"lxml")
    players = [tr.find("th",{"scope":"row"}) for tr in data.findAll("tr",{"class":None})]
    stats = [tr.findAll("td") for tr in data.findAll("tr",{"class":None})]
    players.pop(0)
    stats.pop(0)
    return players,stats

# utiliy method to dynamically scrape tables
def get_pfr_table(link,table):
    page_soup = get_soup(link)
    gameid = link.split('boxscores/')[1].split('.')[0]
    metrics = [TableColumns().football_ref[table]]
    players,stats = get_data(page_soup,table,1)
    for player,stat in zip(players,stats):
        mets = np.array([gameid,player.a['href'],player.text])
        for idx,metric in enumerate(stat):
            value = metric.text
            if not value:
                value = 0
            if idx > 0 and '%' not in metric.text:
                value = float(value)
            mets = np.append(mets,value)
        metrics.append(mets)
    df = np.vstack(metrics)
    return pd.DataFrame(data=df[1:,1:],index=df[1:,0],columns=df[0,1:])

def get_total_offense(link):
    return get_pfr_table(link,'all_player_offense')

def get_receiving(link):
    return get_pfr_table(link,'all_targets_directions')

def get_rushing(link):
    return get_pfr_table(link,'all_rush_directions')

def get_defense(link):
    return get_pfr_table(link,'all_player_defense')

def get_returns(link):
    return get_pfr_table(link,'all_returns')

def get_home_snapcounts(link):
    return get_pfr_table(link,'all_home_snap_counts')

def get_vis_snapcounts(link):
    return get_pfr_table(link,'all_vis_snap_counts')

# for debugging
def get_single_gameinfo(link):
    page_soup = get_soup(link)
    link_comps = link.split('boxscores/')
    link_comps = link_comps[1].split('.')
    gameid = link_comps[0]
    date = gameid[:8]

    # get team names
    gameteams = page_soup.findAll("a",{"itemprop":"name"})
    team_home = gameteams[0].text
    team_away = gameteams[1].text

    # get vegas odds
    gameinfo = page_soup.find("div",{"id":"all_game_info"})
    comment = gameinfo.find(string=lambda text:isinstance(text,Comment))
    gameinfo = soup(comment,"lxml")
    gameinfo = [tr.findAll("td") for tr in gameinfo.findAll("tr",{"class":None})]
    vegasline = gameinfo[-2][0].text.strip()
    vegasline = re.split("\s-",vegasline)
    print(vegasline)
    home_fav = 1
    if vegasline[0] == team_away:
        home_fav = 0
    vegasline = float(vegasline[1])
    overunder = float(gameinfo[-1][0].text.split(" ")[0].strip())
    print('home_fav = ',home_fav,', ats = ',vegasline,', ou = ',overunder)

    # get score
    score = page_soup.findAll("div",{"class":"score"})
    points_home = float(score[0].text.strip())
    points_away = float(score[1].text.strip())
    home_score_diff = points_home - points_away
    print('score_diff = ',home_score_diff)

    # calculate vegas results
    ats_result = 0
    if((home_score_diff > vegasline and home_fav == 1) or (home_score_diff < (vegasline*-1) and home_fav == 0)):
        ats_result = 1
    elif((home_score_diff < vegasline and home_fav == 1) or (home_score_diff > (vegasline*-1) and home_fav == 0)):
        ats_result = -1   

    ou_result = 0
    if(points_home+points_away > overunder):
        ou_result = 1
    elif(points_home+points_away < overunder):
        ou_result = -1

    # return metrics in numpy array
    stats = np.array([gameid,date,team_home,points_home,team_away,points_away,home_fav,vegasline,overunder,ats_result,ou_result])
    return stats

def get_weekly_summary(link):
    page_soup = get_soup(link)
    # get game links for week
    games = page_soup.findAll("td",{"class":"gamelink"})
    all_games = [['gameid','date','team_home','points_home','team_away','points_away','home_fav','vegasline','overunder','ats_result','ou_result']]

    # build dataframe for every game for that week
    for game in games:
        # get date, page soup for game link
        gameid = (str(game.a['href']))
        date = gameid[11:20]
        link = "https://www.pro-football-reference.com"+gameid
        gameinfo = np.array([gameid,date])
        page_soup = get_soup(link)

        # get team names
        gameteams = page_soup.findAll("a",{"itemprop":"name"})
        team_home = gameteams[0].text
        team_away = gameteams[1].text

        # get vegas odds
        gameinfo = page_soup.find("div",{"id":"all_game_info"})
        comment = gameinfo.find(string=lambda text:isinstance(text,Comment))
        gameinfo = soup(comment,"lxml")
        gameinfo = [tr.findAll("td") for tr in gameinfo.findAll("tr",{"class":None})]
        vegasline = gameinfo[-2][0].text.strip()
        home_fav = 1
        if vegasline == 'Pick':
            vegasline = 0
        else:
            vegasline = re.split("\s-",vegasline)
            if vegasline[0] == team_away:
                home_fav = 0
            vegasline = float(vegasline[1])
        overunder = float(gameinfo[-1][0].text.split(" ")[0].strip())

        # get score
        score = page_soup.findAll("div",{"class":"score"})
        points_home = float(score[0].text.strip())
        points_away = float(score[1].text.strip())
        home_score_diff = points_home - points_away

        # calculate vegas results
        ats_result = 0
        if((home_score_diff > vegasline and home_fav == 1) or (home_score_diff < (vegasline*-1) and home_fav == 0)):
            ats_result = 1
        elif((home_score_diff < vegasline and home_fav == 1) or (home_score_diff > (vegasline*-1) and home_fav == 0)):
            ats_result = -1   

        ou_result = 0
        if(points_home+points_away > overunder):
            ou_result = 1
        elif(points_home+points_away < overunder):
            ou_result = -1

        # return metrics in numpy array
        stats = np.array([gameid,date,team_home,points_home,team_away,points_away,home_fav,vegasline,overunder,ats_result,ou_result])
        all_games.append(stats)
    df = np.vstack(all_games)
    return pd.DataFrame(data=df[1:,1:],index=df[1:,0],columns=df[0,1:])

In [22]:
class PFRScraper():
    def __init__(self,season,week):
        self.season = season
        self.week = week
        self.weekly_link = "https://www.pro-football-reference.com/years/"+str(season)+"/week_"+str(week)+".htm"
        self.week_soup = get_soup(self.weekly_link)
        self.games = self.week_soup.findAll("td",{"class":"gamelink"})
        self.tables = {
            'weekly_summary':get_weekly_summary(self.weekly_link)
        }
        print("Weekly summary for ",season,'-',week,' successfully retrieved.')
        for game in self.games:
            gametable = {}
            gameid = (str(game.a['href']))
            link = "https://www.pro-football-reference.com"+gameid
            print("Scraping stats from ",link)
            gameid = link.split('boxscores/')[1].split('.')[0]
            gametable['link'] = link
            gametable['offense'] = get_total_offense(link)
#             gametable['receiving'] = get_receiving(link)
#             gametable['rushing'] = get_rushing(link)
#             gametable['defense'] = get_defense(link)
#             gametable['returns'] = get_returns(link)
#             gametable['home_snaps'] = get_home_snapcounts(link)
#             gametable['away_snaps'] = get_vis_snapcounts(link)
            self.tables[gameid] = gametable

In [23]:
for i in range(2009,2019):
    print("Getting stats for ",i)
    for j in range(1,18):
        print("Parsing week ",j)
        pf_scraper = PFRScraper(i,j)

Getting stats for  2009
Parsing week  1
Weekly summary for  2009 - 1  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909100pit.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130cin.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130rav.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130clt.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130tam.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130nor.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130htx.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130cle.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200909130atl.htm
Scraping stats from  https://www.pro

Parsing week  7
Weekly summary for  2009 - 7  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250ram.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250kan.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250cle.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250pit.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250htx.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250tam.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250rai.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250dal.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200910250mia.htm
Scraping stats from  https://www.pro-football-reference.com/

Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060pit.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060mia.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060cin.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060atl.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060was.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060jax.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060clt.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060cle.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060nyg.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/200912060sea.htm
Scraping stats from  https://www.pro-footba

Scraping stats from  https://www.pro-football-reference.com/boxscores/201009120was.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009130nyj.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009130kan.htm
Parsing week  2
Weekly summary for  2010 - 2  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190cle.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190cin.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190oti.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190gnb.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190det.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201009190atl.htm
Scraping stats from  https://www.pro-football-reference.com/

Scraping stats from  https://www.pro-football-reference.com/boxscores/201010250dal.htm
Parsing week  8
Weekly summary for  2010 - 8  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310ram.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310det.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310nyj.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310dal.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310cin.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310sfo.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310kan.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310sdg.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201010310crd.htm
Scraping stats from  https://www.pro-football-reference.com/

...

Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290buf.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290tam.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290phi.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290nyj.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290nwe.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290nor.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290sea.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290was.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710290det.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201710300kan.htm
Parsing week  9
Weekly summary for  2017 - 9  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/

Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100htx.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100cle.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100sdg.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100den.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100crd.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100ram.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100jax.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712100pit.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712110mia.htm
Parsing week  15
Weekly summary for  2017 - 15  successfully retrieved.
Scraping stats from  https://www.pro-football-reference.com/boxscores/201712140clt.htm
Scraping stats from  https://www.pro-football-reference.co

Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230atl.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230was.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230rav.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230phi.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230min.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230mia.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230kan.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230jax.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230htx.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201809230ram.htm
Scraping stats from  https://www.pro-footba

Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040car.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040buf.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040was.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040rav.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040min.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040mia.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040sea.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040den.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040nor.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811040nwe.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201811050dal.htm
Parsing week  10
Weekly summary for  2018 -

Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160nyg.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160min.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160jax.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160clt.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160cin.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160chi.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160buf.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160sfo.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160pit.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812160ram.htm
Scraping stats from  https://www.pro-football-reference.com/boxscores/201812170car.htm
Parsing week  16
Weekly summary for  2018 -